# Algoritmus SIMPLE pro Navierovy-Stokesovy rovnice

In [1]:
import Base.+, Base.-, Base.*
using PyPlot

In [2]:
include("cartesianmesh.jl");

In [3]:
type Field{T}
    value 
    mesh
    boundarycondition
end

In [26]:
ScalarField(m :: CartesianMesh, bc) = Field{Float64}(zeros(m.nx*m.ny), m, bc);
VectorField(m :: CartesianMesh, bc) = Field{Vec2d}(zeros(Vec2d, m.nx*m.ny,2), m, bc);

In [27]:
asmatrix(f::Field{Float64}) = reshape(f.value, (f.mesh.nx,f.mesh.ny) );

function asmatrix(f::Field{Vec2d})
    m = zeros(f.mesh.nx, f.mesh.ny, 2)
    for j=1:f.mesh.ny, i=1:f.mesh.nx
        m[i,j,:] = f.value[i+(j-1)*f.mesh.nx]
    end
    return m
end;

In [28]:
type Equation{T}
    A
    x
    b
end

LoadError: invalid redefinition of constant Equation

In [29]:
-(eq::Equation) = Equation(-eq.A, eq.x, -eq.b);

-{T}(eq::Equation{T}, b::Array{T,1}) = Equation(eq.A, eq.x, eq.b + b);

+{T}(eq::Equation{T}, b::Array{T,1}) = Equation{T}(eq.A, eq.x, eq.b - b);

*{T}(a::Float64, eq::Equation{T}) = Equation{T}(a*eq.A, eq.x, a*eq.b);

In [30]:
Ac(eq::Equation) = diag(eq.A);

function H(eq::Equation)
    res = copy(eq.b)
    I,J,V = findnz(eq.A)
    for ijv in zip(I,J,V)
        i,j,v = ijv
        if i!=j
            eq.b[i] -= v * eq.x[i]
        end
    end
    return res
end

H (generic function with 1 method)

In [31]:
function relax!(eqn::Equation, α)
    D = diag(eqn.A)
    for i=1:length(D) 
        eqn.A[i,i] /= α
    end
    eqn.b += (1-α)/α * D .* eqn.x
end;

In [32]:
function solve!(eqn::Equation{Float64})
    x = eqn.A \ eqn.b
    for i in eachindex(eqn.x)
        eqn.x[i] = x[i]
    end
end;

In [51]:
function solve!(eqn::Equation{Vec2d})
    b = zeros(length(eqn.b),2)
    for i=1:length(eqn.b)
        b[i,:] = eqn.b[i]
    end
    println(eqn.b)
    println(size(eqn.b))
    println(size(b))
    
    x = eqn.A \ b

    for i in eachindex(eqn.x)
        eqn.x[i] = x[i,:]
    end

end;

In [52]:
type Dirichlet
    value
end

type Neumann
    value
end

bndvalue(uin, Δ, bc::Dirichlet) = bc.value;
bndvalue(uin, Δ, bc::Neumann) = uin + Δ * bc.value;

ddncoeffs(Δ, bc::Dirichlet) = (-1/Δ, bc.value/Δ);
ddncoeffs(Δ, bc::Neumann) = (0, bc.value);

In [53]:
function createfields(msh)
    U = VectorField(msh, Dict( 
            "top"=>Dirichlet(Vec2d(1,0)), 
            "left"=>Dirichlet(Vec2d(0,0)), 
            "right"=>Dirichlet(Vec2d(0,0)), 
            "bottom"=>Dirichlet(Vec2d(0,0)) 
            ));
        
    p = ScalarField(msh, Dict( 
            "top"=>Neumann(0), 
            "left"=>Neumann(0), 
            "right"=>Neumann(0), 
            "bottom"=>Neumann(0)
            ));
    return (U,p)
end

createfields (generic function with 1 method)

In [54]:
function laplace{T}(ν, u::Field{T})
    mesh = u.mesh
    dims = (mesh.nx, mesh.ny)
    n  = prod(dims)
    A = spzeros(n,n)
    b = zeros(u.value)

    
    for f in internalfaces(mesh)
        owner, neigh = f.owner, f.neigh
        
        νf = (ν[owner]+ν[neigh]) / 2.0
        
        co = cell(mesh, owner)
        cn = cell(mesh, neigh)
        
        g = νf * norm(f.s) / norm(cn.x-co.x)

        A[owner, owner] -= g / co.vol
        A[owner, neigh] += g / co.vol
            
        A[neigh, owner] += g / cn.vol
        A[neigh, neigh] -= g / cn.vol
    end
    
    for patch in patches(mesh)

        bc = u.boundarycondition[ name(patch) ]
        for f in boundaryfaces(patch)
            owner = f.owner
            co = cell(mesh, owner)
            
            Δ = norm(f.x - co.x)
            νf = ν[owner]
            
            a,val = ddncoeffs(Δ, bc)
            A[owner,owner] += νf * a * norm(f.s) / co.vol
            b[owner] -= νf * val * norm(f.s) / co.vol
        end
    end
    
    Equation{T}(A, u.value, b)
end

laplace(u) = laplace(ones(u.mesh.nx*u.mesh.ny), u)

laplace (generic function with 2 methods)

In [55]:
m3 = CartesianMesh(3,3);
U,p = createfields(m3);

In [56]:
uEqn = laplace(U);
solve!(uEqn)

StaticArrays.SVector{2,Float64}[[0.0,0.0] [0.0,0.0]; [0.0,0.0] [0.0,0.0]; [0.0,0.0] [0.0,0.0]; [0.0,0.0] [0.0,0.0]; [0.0,0.0] [0.0,0.0]; [0.0,0.0] [0.0,0.0]; [-18.0,0.0] [0.0,0.0]; [-18.0,0.0] [0.0,0.0]; [-18.0,0.0] [0.0,0.0]]
(9,2)
(18,2)


LoadError: DimensionMismatch("LHS and RHS should have the same number of rows. LHS has 9 rows, but RHS has 18 rows.")

In [17]:
asmatrix(U)[:,:,1]

LoadError: DimensionMismatch("new dimensions (3,3) must be consistent with array size 18")

In [18]:
function grad(p::Field{Float64})
    mesh = p.mesh
    dp = zeros(Vec2d,length(p.value))
    
    for f in internalfaces(mesh)
        owner = f.owner
        neigh = f.neigh
        
        pf = (p.value[owner] + p.value[neigh]) / 2.0
        
        dp[owner] += pf * f.s
        dp[neigh] -= pf * f.s
    end

    for patch in patches(mesh)
        bc = p.boundarycondition[ name(patch) ]
        for f in boundaryfaces(patch)
            owner = f.owner
            co = cell(mesh, owner)
            Δ = norm(f.x - co.x)
            pf = bndvalue(p.value[owner], Δ, bc)
            dp[owner] += pf * f.s
        end
    end
    
    for c in cells(mesh)
        dp[c.id] /= c.vol
    end
    
    return dp
end;

## Stokesuv problem (test)

In [19]:
ν = 0.01;
msh25 = CartesianMesh(25,25);

In [25]:
U,p = createfields(msh25);

α = 0.7
β = 0.3

for iter = 0:0
    
    UOld, pOld = copy(U.value), copy(p.value)
    
    UEqn = (-ν)*laplace(U)

    relax!(UEqn, α)
    
    solve!(UEqn + grad(p))
    
    #ra = 1 ./ Ac(UEqn);
    
    #UBar = Field{Vec2d}( ra .* H(UEqn), U.mesh, U.boundarycondition);
    
    #pEqn = laplace(ra, p) - (ddx(uBar) + ddy(vBar));
    #pEqn.A[1,1] -= 1/(p.mesh.Δx*p.mesh.Δy)
    #pEqn.A[1,1] *= 2
    #solve!(pEqn)
    
    #p.value = (1-β) * pOld + β * p.value
    #U.value = UBar.value - ra .* grad(p)
    
    #if rem(iter,5)==0
    #    nxny = u.mesh.nx*u.mesh.ny
    #    pRez = norm(pOld - p.value) / nxny
    #    println(iter, "\t", pRez)
    #end
end

LoadError: MethodError: Cannot `convert` an object of type StaticArrays.SVector{2,Float64} to an object of type Float64
This may have arisen from a call to the constructor Float64(...),
since type constructors fall back to convert methods.